# Classifier tests

In [12]:
import os
import cv2
import random
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
from matplotlib import pyplot as plt
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.face_detector import FaceDetector
from services.image_editor import ImageEditor

model = YOLO("weights/yolov8_classification_model.pt").to("cuda")
face_detector = FaceDetector()

Face detector warmed up.


In [66]:
def random_image(root):
    images_paths = []

    for root, _, files in os.walk(root):
        for file in files:
            images_paths.append(os.path.join(root, file))

    return random.choice(images_paths) if images_paths else None

def get_random_face_path(person_name):
    root_dir = "training\\classifier\\dataset\\train"
    person_dir = os.path.join(root_dir, person_name)
    if not os.path.exists(person_dir):
        return "Person directory does not exist."

    images = os.listdir(person_dir)
    if not images:
        return "No images found for the person."

    random_image = random.choice(images)
    return os.path.join(person_dir, random_image)

def cosine_distance(features1, features2) -> float:
    a = np.matmul(np.transpose(features1), features2)
    b = np.sum(np.multiply(features1, features1))
    c = np.sum(np.multiply(features2, features2))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def get_person_median_distance(person_folder_path):
    compared_pairs = set()
    distances: [float] = []
    name = person_folder_path.replace("\\", "/").rpartition("/")[-1]

    if os.path.isdir(person_folder_path):
        for image1 in os.listdir(person_folder_path):
            for image2 in os.listdir(person_folder_path):
                pair = tuple(sorted([image1, image2]))
                if pair not in compared_pairs and image1 != image2:
                    is_compared, dst = compare(os.path.join(person_folder_path, image1), os.path.join(person_folder_path, image2))
                    if is_compared:
                        print("{}: Comparison {} and {}: dst {}".format(name, image1, image2, dst))
                        distances.append(dst)
                        compared_pairs.add(pair)
    
    distances.sort()
    median = distances[len(distances) // 2]
    return median

def compare(image1_path, image2_path):
    img1 = cv2.cvtColor(cv2.imread(image1_path), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(image2_path), cv2.COLOR_BGR2RGB)
    pred1 = face_detector.detect(img1)
    pred2 = face_detector.detect(img2)
    if len(pred1) == 0 or len(pred2) == 0:
        return False, 0
    img1 = ImageEditor.crop(img1, pred1[0].bounding_box)
    img2 = ImageEditor.crop(img2, pred2[0].bounding_box)
    features1 = model.predict(img1, verbose=False)[0].cpu().tolist()
    features2 = model.predict(img2, verbose=False)[0].cpu().tolist()
    return True, cosine_distance(features1, features2)

In [ ]:
aj1_path = get_random_face_path("Angelina Jolie")
aj2_path = get_random_face_path("Angelina Jolie")

bp1_path = get_random_face_path("Brad Pitt")
bp2_path = get_random_face_path("Brad Pitt")

hj1_path = get_random_face_path("Hugh Jackman")
hj2_path = get_random_face_path("Hugh Jackman")

aj1 = cv2.cvtColor(cv2.imread(aj1_path), cv2.COLOR_BGR2RGB)
aj2 = cv2.cvtColor(cv2.imread(aj2_path), cv2.COLOR_BGR2RGB)
bp1 = cv2.cvtColor(cv2.imread(bp1_path), cv2.COLOR_BGR2RGB)
bp2 = cv2.cvtColor(cv2.imread(bp2_path), cv2.COLOR_BGR2RGB)
hj1 = cv2.cvtColor(cv2.imread(hj1_path), cv2.COLOR_BGR2RGB)
hj2 = cv2.cvtColor(cv2.imread(hj2_path), cv2.COLOR_BGR2RGB)

aj1 = ImageEditor.crop(aj1, face_detector.detect(aj1)[0].bounding_box)
aj2 = ImageEditor.crop(aj2, face_detector.detect(aj2)[0].bounding_box)
bp1 = ImageEditor.crop(bp1, face_detector.detect(bp1)[0].bounding_box)
bp2 = ImageEditor.crop(bp2, face_detector.detect(bp2)[0].bounding_box)
hj1 = ImageEditor.crop(hj1, face_detector.detect(hj1)[0].bounding_box)
hj2 = ImageEditor.crop(hj2, face_detector.detect(hj2)[0].bounding_box)

tested_image_1 = aj1
tested_image_2 = hj1

plt.subplot(1, 2, 1)
plt.imshow(tested_image_1)
plt.subplot(1, 2, 2)
plt.imshow(tested_image_2)
plt.show()

features1 = model.predict(tested_image_1, verbose=False)
features2 = model.predict(tested_image_2, verbose=False)

df1_embedding = DeepFace.represent(
                img_path=tested_image_1,
                model_name="VGG-Face",
                enforce_detection=False,
                detector_backend="skip",
                align=True,
                normalization="base",
            )
df2_embedding = DeepFace.represent(
                img_path=tested_image_2,
                model_name="VGG-Face",
                enforce_detection=False,
                detector_backend="skip",
                align=True,
                normalization="base",
            )

img1_representation = df1_embedding[0]["embedding"]
img2_representation = df2_embedding[0]["embedding"]

features1_list = features1[0].cpu().tolist()
features2_list = features2[0].cpu().tolist()

res = cosine_distance(features1_list, features2_list)
res2 = cosine_distance(img1_representation, img2_representation)
print("RGB: Yolov8-cls", res, "- DeepFace", res2)

# Get all medians

In [ ]:
distances = []
dataset_root = "training\\classifier\\dataset\\train"
for root, dirs, _ in os.walk(dataset_root):
    for dir in dirs:
        dst = get_person_median_distance(os.path.join(root, dir))
        distances.append([dir, dst])
        print("Median distance for {}: {}".format(dir, dst))

In [ ]:
distances.sort(key=lambda x: x[1])
average_distance = sum([distance[1] for distance in distances]) / len(distances)
mediane = distances[len(distances) // 2][1]

print(f"Average distance: {average_distance} - Median distance: {mediane}")